In [99]:
s = "babad"
res = ''
def check(left, right):
    while left >= 0 and right < len(s) and s[left] == s[right]:
        left -= 1
        right += 1
    return s[left+1:right]

for i in range(len(s)-1):
    res = max(res, check(i,i+1), check(i,i+2), key = len)
res

'bab'